In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from toolz.curried import *

In [2]:
import sys
sys.path.append('/Users/henriquelopes/Projects/cartolaAPI')

In [ ]:
from data_science_utils.feature_creation.filters import *
from data_science_utils.feature_creation.getters import *
from data_science_utils.feature_creation.aggregators import *

In [ ]:
from data_science_utils.feature_creation.team_features import *
from data_science_utils.feature_creation.position_features import *
from data_science_utils.feature_creation.individual_features import *

In [ ]:
## Constants
abreviacao = ['RB', 'FC', 'GC', 'CA', 'CV', 'SG', 'DD', 'DP', 'GS',
              'FS', 'PE', 'A', 'FT', 'FD', 'FF', 'G', 'I', 'PP']
descricao = ['Robadas de Bola', 'Faltas Cometidas', 'Gol Contra', 'Cartao Amarelo',
             'Cartao Vermelho', 'Jogo sem sofrer gol', 'Defesa Dificil', 'Defesa de Penalti',
             'Gol Sofrido', 'Falta Sofrida', 'Passe Errado', 'Assistencia',
             'Finalizacao na Trave', 'Finalizacao defendida', 'Finalizao pra Fora', 'Gol',
             'Impedimento', 'Penalti Perdido']
pontuacao = [1.7, -0.5, -6.0, -2.0, -5.0, 5.0, 3.00, 7.00, -2.0,
             0.5, -0.3, 5.0, 3.5, 1.0, 0.7, 8.0, -0.5, -3.5]

points_dict = {
    "RB": 1.7,
    "FC": -0.5,
    "GC": -6.0,
    "CA": -2.0,
    "CV": -5.0,
    "SG": 5.0,
    "DD": 3.0,
    "DP": 7.0,
    "GS": -2.0,
    "FS": 0.5,
    "PE": -0.3,
    "A": 5.0,
    "FT": 3.5,
    "FD": 1.0,
    "FF": 0.7,
    "G": 8.0,
    "I": -0.5,
    "PP": -3.5,
}

team_dict = {
'265':'BAH',
'288':'CRI',
'267':'VAS',
'290':'GOI',
'314':'AVA',
'317':'JEC',
'262':'FLA',
'263':'BOT',
'264':'COR',
'266':'FLU',
'275':'PAL',
'276':'SAO',
'277':'SAN',
'282':'CAM',
'283':'CRU',
'284':'GRE',
'285':'INT',
'287':'VIT',
'292':'SPT',
'293':'CAP',
'294':'CFC',
'303':'PON',
'315':'CHA',
'316':'FIG',
'327':'AME',
'344':'SCZ',
'373':'ATL-GO'
}

posicoes = ["gol", "lat", "zag",  "mei", "ata", "tec"]

status = {
		"2": "Dúvida",
		"3": "Suspenso",
		"5": "Contundido",
		"6": "Nulo",
		"7": "Provável"
	}


In [ ]:
df = pd.read_csv('../db/cartola_aggregated_with_results.csv').rename(columns={'Unnamed: 0': 'ScoutID'})

In [ ]:
df.sort_values(['AtletaID', 'ano', 'Rodada'], inplace=True)

In [ ]:
df = df.reset_index(drop=True)

In [ ]:
df.head()

In [ ]:
df['is_next_same'] = df['AtletaID'] == df['AtletaID'].shift(-1)

In [ ]:
df['target'] = df['Pontos'].shift(-1)
df.loc[~(df['is_next_same']), 'target'] = -999 

In [ ]:
df['game_points'] = df.apply(lambda x: 3 if ((x['result'] == 1 and x['Home']) or (x['result'] == -1 and not x['Home']))
                        else 1 if x['result'] == 0 else 0, axis=1)

In [ ]:
df.drop(['pred.home.score', 'pred.away.score', 'home.attack', 'home.defend', 'variable'],
        axis=1, inplace=True)

In [ ]:
df.drop(['avg.Points', 'avg.last05', 'avg.FS', 'avg.FS.l05', 
         'avg.PE', 'avg.PE.l05', 'avg.A', 'avg.A.l05', 'avg.FT', 
         'avg.FT.l05', 'avg.FD', 'avg.FD.l05', 'avg.FF', 'avg.FF.l05', 
         'avg.G', 'avg.G.l05', 'avg.I', 'avg.I.l05', 'avg.PP', 'avg.PP.l05',
         'avg.RB', 'avg.RB.l05', 'avg.FC', 'avg.FC.l05', 'avg.GC', 'avg.GC.l05',
         'avg.CA', 'avg.CV.l05', 'avg.SG', 'avg.SG.l05',
         'avg.DD', 'avg.DD.l05', 'avg.DP', 'avg.DP.l05', 'avg.GS', 'avg.GS.l05', 'risk_points'], axis=1, inplace=True)

In [4]:
columns_to_average = ['FS', 'PE', 'A','FT', 'FD', 'FF', 'G', 'I', 'PP', 'RB', 'FC', 
 'GC', 'CA', 'CV', 'SG','DD', 'DP', 'GS', 'Pontos', 'Preco', 'game_points']

In [3]:
def groupbyTeam(df, aggCols):
    return df.groupby(['ClubeID', 'ano', 'Rodada'], as_index=False)[aggCols].mean()

def groupbyPos(df, aggCols):
    return df.groupby(['Posicao', 'ano', 'Rodada'], as_index=False)[aggCols].mean()

def filterHome(df, home):
    return df.loc[(df.Home == home)].reset_index(drop=True)

In [ ]:
print(list(df.columns))

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
auxiliary = ['AtletaID', 'Rodada', 'ClubeID', 'Participou', 'Posicao', 'ano', 'mes', 'dia', 'Apelido', 'Status']

In [ ]:
df.Jogos.fillna(0, inplace=True)

In [ ]:
df.head()

In [ ]:
df[['ClubeID', 'Jogos']].isna().sum()

In [ ]:
df[['ClubeID', 'Jogos']] = df[['ClubeID', 'Jogos']].astype('Int64')

In [ ]:
df.dtypes

In [ ]:
df = df.loc[(df.Posicao != 'tec')]

In [5]:
def getWindowOver(df, overCol, groupCols, aggCols, window_size, na_value, suffix):
    return (df.set_index(overCol).groupby(groupCols, as_index=False)[aggCols]
            .rolling(5, min_periods=0).mean().fillna(na_value)
            .reset_index(drop=True)
            .rename(columns=merge(map(lambda name: {name: name + suffix + '_last_' + str(window_size)}, aggCols))))

In [ ]:
df.head()

In [ ]:
df_player_avg = pd.DataFrame()
for num_rodadas in (5,10,20):
    df_player_avg = pd.concat([getWindowOver(df, 'Rodada', ['AtletaID', 'ano'], columns_to_average,
                                         num_rodadas, 0, '_avg'), df_player_avg], axis=1)

In [ ]:
df_player = pd.concat([df, df_player_avg], axis=1)

In [ ]:
df[['Pontos', 'Pontos_avg_last_5']].head()

In [ ]:
df_player_home_avg = pd.DataFrame()
for num_rodadas in (5,10,20):
    df_player_home_avg = pd.concat([getWindowOver(filterHome(df, True).reset_index(drop=True), 'Rodada', ['AtletaID', 'ano'], columns_to_average,
                                         num_rodadas, 0, '_avg_home'), df_player_home_avg], axis=1)

In [ ]:
df_player_home_avg['Pontos_avg_home_last_5'].head()

In [ ]:
df_home = pd.concat([filterHome(df, True).reset_index(drop="True"), df_player_home_avg], axis=1)

In [ ]:
df_home[['Pontos', 'Pontos_avg_home_last_5']].head()

In [ ]:
print(list(df_home.columns))

In [ ]:
home_columns=['AtletaID', 'Rodada', 'ano', 'FS_avg_home_last_20', 'PE_avg_home_last_20', 'A_avg_home_last_20', 'FT_avg_home_last_20', 'FD_avg_home_last_20', 'FF_avg_home_last_20', 'G_avg_home_last_20', 'I_avg_home_last_20', 'PP_avg_home_last_20', 'RB_avg_home_last_20', 'FC_avg_home_last_20', 'GC_avg_home_last_20', 'CA_avg_home_last_20', 'CV_avg_home_last_20', 'SG_avg_home_last_20', 'DD_avg_home_last_20', 'DP_avg_home_last_20', 'GS_avg_home_last_20', 'Pontos_avg_home_last_20', 'Preco_avg_home_last_20', 'game_points_avg_home_last_20', 'FS_avg_home_last_10', 'PE_avg_home_last_10', 'A_avg_home_last_10', 'FT_avg_home_last_10', 'FD_avg_home_last_10', 'FF_avg_home_last_10', 'G_avg_home_last_10', 'I_avg_home_last_10', 'PP_avg_home_last_10', 'RB_avg_home_last_10', 'FC_avg_home_last_10', 'GC_avg_home_last_10', 'CA_avg_home_last_10', 'CV_avg_home_last_10', 'SG_avg_home_last_10', 'DD_avg_home_last_10', 'DP_avg_home_last_10', 'GS_avg_home_last_10', 'Pontos_avg_home_last_10', 'Preco_avg_home_last_10', 'game_points_avg_home_last_10', 'FS_avg_home_last_5', 'PE_avg_home_last_5', 'A_avg_home_last_5', 'FT_avg_home_last_5', 'FD_avg_home_last_5', 'FF_avg_home_last_5', 'G_avg_home_last_5', 'I_avg_home_last_5', 'PP_avg_home_last_5', 'RB_avg_home_last_5', 'FC_avg_home_last_5', 'GC_avg_home_last_5', 'CA_avg_home_last_5', 'CV_avg_home_last_5', 'SG_avg_home_last_5', 'DD_avg_home_last_5', 'DP_avg_home_last_5', 'GS_avg_home_last_5', 'Pontos_avg_home_last_5', 'Preco_avg_home_last_5', 'game_points_avg_home_last_5']

In [ ]:
df_player = df_player.merge(df_home[home_columns], how="left", on=['AtletaID', 'Rodada', 'ano'])

In [ ]:
df_player_away_avg = pd.DataFrame()
for num_rodadas in (5,10,20):
    df_player_away_avg = pd.concat([getWindowOver(filterHome(df, False),
                                                  'Rodada', ['AtletaID', 'ano'],
                                                  columns_to_average,num_rodadas, 0, '_avg_away'),
                                    df_player_away_avg], axis=1)

In [ ]:
df_away = pd.concat([filterHome(df, False), df_player_away_avg], axis=1)

In [ ]:
df_away[['Pontos', 'Pontos_avg_away_last_5']].head()

In [ ]:
print(list(df_away.columns))

In [ ]:
away_cols = ['AtletaID','Rodada', 'ano', 'FS_avg_away_last_20', 'PE_avg_away_last_20', 'A_avg_away_last_20', 'FT_avg_away_last_20', 'FD_avg_away_last_20', 'FF_avg_away_last_20', 'G_avg_away_last_20', 'I_avg_away_last_20', 'PP_avg_away_last_20', 'RB_avg_away_last_20', 'FC_avg_away_last_20', 'GC_avg_away_last_20', 'CA_avg_away_last_20', 'CV_avg_away_last_20', 'SG_avg_away_last_20', 'DD_avg_away_last_20', 'DP_avg_away_last_20', 'GS_avg_away_last_20', 'Pontos_avg_away_last_20', 'Preco_avg_away_last_20', 'game_points_avg_away_last_20', 'FS_avg_away_last_10', 'PE_avg_away_last_10', 'A_avg_away_last_10', 'FT_avg_away_last_10', 'FD_avg_away_last_10', 'FF_avg_away_last_10', 'G_avg_away_last_10', 'I_avg_away_last_10', 'PP_avg_away_last_10', 'RB_avg_away_last_10', 'FC_avg_away_last_10', 'GC_avg_away_last_10', 'CA_avg_away_last_10', 'CV_avg_away_last_10', 'SG_avg_away_last_10', 'DD_avg_away_last_10', 'DP_avg_away_last_10', 'GS_avg_away_last_10', 'Pontos_avg_away_last_10', 'Preco_avg_away_last_10', 'game_points_avg_away_last_10', 'FS_avg_away_last_5', 'PE_avg_away_last_5', 'A_avg_away_last_5', 'FT_avg_away_last_5', 'FD_avg_away_last_5', 'FF_avg_away_last_5', 'G_avg_away_last_5', 'I_avg_away_last_5', 'PP_avg_away_last_5', 'RB_avg_away_last_5', 'FC_avg_away_last_5', 'GC_avg_away_last_5', 'CA_avg_away_last_5', 'CV_avg_away_last_5', 'SG_avg_away_last_5', 'DD_avg_away_last_5', 'DP_avg_away_last_5', 'GS_avg_away_last_5', 'Pontos_avg_away_last_5', 'Preco_avg_away_last_5', 'game_points_avg_away_last_5']

In [ ]:
df_player = df_player.merge(df_away[away_cols], how="left", on=['AtletaID', 'Rodada', 'ano'])

In [ ]:
for x,y in list(zip(home_columns, away_cols)):
    df_player[x].fillna(df_player[y], inplace=True)

In [ ]:
df_player.drop(list(set(away_cols) - set(['AtletaID', 'Rodada', 'ano'])),
        axis=1, inplace=True)

In [ ]:
df_player.to_csv('../db/features_jogadores_final_calculadas.csv')

In [49]:
df_player = pd.read_csv('../db/features_jogadores_final_calculadas.csv')

/Users/henriquelopes/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (5,39,41) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
df_player.shape

In [ ]:
df_player.head()

In [ ]:
df_player[home_columns].head()

In [ ]:
print(list(df_player.columns))

In [50]:
df_team = df_player.groupby(['ClubeID', 'ano', 'Rodada'], as_index=False)[columns_to_average].mean()

In [ ]:
df_team.head()

In [ ]:
df_team.shape

In [51]:
df_team_avg = pd.DataFrame()
for num_rodadas in (5,10,20):
    df_team_avg = pd.concat([getWindowOver(groupbyTeam(df_player, columns_to_average), 'Rodada', ['ClubeID', 'ano'], columns_to_average,
                                         num_rodadas, 0, '_avg_team'), df_team_avg], axis=1)

In [ ]:
df_team_avg.head()

In [ ]:
df_team.shape

In [52]:
df_team = pd.concat([df_team, df_team_avg], axis=1)

In [53]:
df_team_home_avg = pd.DataFrame()
for num_rodadas in (5,10,20):
    df_team_home_avg = pd.concat([getWindowOver(groupbyTeam(filterHome(df_player, True), columns_to_average),
                                                'Rodada', ['ClubeID', 'ano'], columns_to_average,
                                         num_rodadas, 0, '_avg_team_home'), df_team_home_avg], axis=1)

In [54]:
df_team_home_avg.head()

,FS_avg_team_home_last_20,PE_avg_team_home_last_20,A_avg_team_home_last_20,FT_avg_team_home_last_20,FD_avg_team_home_last_20,FF_avg_team_home_last_20,G_avg_team_home_last_20,I_avg_team_home_last_20,PP_avg_team_home_last_20,RB_avg_team_home_last_20,...,GC_avg_team_home_last_5,CA_avg_team_home_last_5,CV_avg_team_home_last_5,SG_avg_team_home_last_5,DD_avg_team_home_last_5,DP_avg_team_home_last_5,GS_avg_team_home_last_5,Pontos_avg_team_home_last_5,Preco_avg_team_home_last_5,game_points_avg_team_home_last_5
0,0.923077,3.230769,0.000000,0.000000,0.307692,0.230769,0.000000,0.307692,0.0,0.923077,...,0.0,0.307692,0.0,0.307692,0.076923,0.0,0.000000,1.992308,9.200000,1.000000
1,1.153846,2.538462,0.115385,0.000000,0.269231,0.269231,0.153846,0.307692,0.0,1.153846,...,0.0,0.230769,0.0,0.153846,0.038462,0.0,0.076923,3.446154,9.208462,2.000000
2,1.031136,2.882784,0.076923,0.023810,0.298535,0.298535,0.102564,0.324176,0.0,1.221612,...,0.0,0.177656,0.0,0.102564,0.097070,0.0,0.098901,2.971245,9.679212,1.333333
3,1.041209,2.644231,0.075549,0.017857,0.223901,0.277473,0.094780,0.314560,0.0,1.130495,...,0.0,0.186813,0.0,0.076923,0.108516,0.0,0.092033,2.628434,9.648159,1.250000
4,1.063736,2.746154,0.075824,0.014286,0.225275,0.314286,0.091209,0.251648,0.0,1.165934,...,0.0,0.149451,0.0,0.061538,0.132967,0.0,0.089011,2.845824,9.396527,1.200000


In [55]:
df_team_home_avg.shape

(1330, 63)

In [56]:
team_home_cols = ['ClubeID', 'Rodada', 'ano'] + list(df_team_home_avg.columns)

In [57]:
df_home_team = pd.concat([groupbyTeam(filterHome(df_player, True), columns_to_average), df_team_home_avg], axis=1)

In [58]:
df_team_away_avg = pd.DataFrame()
for num_rodadas in (5,10,20):
    df_team_away_avg = pd.concat([getWindowOver(groupbyTeam(filterHome(df_player, False), columns_to_average),
                                                'Rodada', ['ClubeID', 'ano'], columns_to_average,
                                         num_rodadas, 0, '_avg_team_away'), df_team_away_avg], axis=1)

In [59]:
df_team_away_avg.head()

,FS_avg_team_away_last_20,PE_avg_team_away_last_20,A_avg_team_away_last_20,FT_avg_team_away_last_20,FD_avg_team_away_last_20,FF_avg_team_away_last_20,G_avg_team_away_last_20,I_avg_team_away_last_20,PP_avg_team_away_last_20,RB_avg_team_away_last_20,...,GC_avg_team_away_last_5,CA_avg_team_away_last_5,CV_avg_team_away_last_5,SG_avg_team_away_last_5,DD_avg_team_away_last_5,DP_avg_team_away_last_5,GS_avg_team_away_last_5,Pontos_avg_team_away_last_5,Preco_avg_team_away_last_5,game_points_avg_team_away_last_5
0,1.153846,3.307692,0.0,0.000000,0.307692,0.307692,0.0,0.000000,0.0,1.384615,...,0.0,0.076923,0.076923,0.000000,0.076923,0.0,0.153846,1.346154,8.720769,0.000000
1,1.184066,3.046703,0.0,0.000000,0.225275,0.332418,0.0,0.178571,0.0,1.049451,...,0.0,0.181319,0.038462,0.000000,0.074176,0.0,0.148352,0.737363,8.581099,0.000000
2,1.051282,3.126374,0.0,0.047619,0.173993,0.316850,0.0,0.190476,0.0,1.080586,...,0.0,0.192308,0.025641,0.142857,0.097070,0.0,0.098901,1.758242,8.248114,0.333333
3,1.056319,3.076923,0.0,0.035714,0.148352,0.291209,0.0,0.160714,0.0,1.042582,...,0.0,0.162088,0.019231,0.107143,0.144231,0.0,0.127747,1.647253,8.019478,0.250000
4,1.016484,2.975824,0.0,0.028571,0.118681,0.247253,0.0,0.128571,0.0,1.076923,...,0.0,0.143956,0.029670,0.085714,0.129670,0.0,0.159341,1.393516,7.802868,0.200000


In [60]:
df_team_away_avg.shape

(1329, 63)

In [61]:
team_away_cols = ['ClubeID', 'Rodada', 'ano'] + list(df_team_away_avg.columns)

In [62]:
df_away_team = pd.concat([groupbyTeam(filterHome(df_player, False), columns_to_average), df_team_away_avg], axis=1)

In [63]:
df_team = df_team.merge(df_home_team[team_home_cols], how="left", on=['ClubeID', 'Rodada', 'ano'])

In [64]:
df_team.shape

(2659, 150)

In [65]:
df_team = df_team.merge(df_away_team[team_away_cols], how="left", on=['ClubeID', 'Rodada', 'ano'])

In [66]:
for x,y in list(zip(team_home_cols, team_away_cols)):
    df_team[x].fillna(df_team[y], inplace=True)

In [67]:
df_team.drop(list(set(team_away_cols) - set(['ClubeID', 'Rodada', 'ano'])),
        axis=1, inplace=True)

In [68]:
df_team.head()

,ClubeID,ano,Rodada,FS,PE,A,FT,FD,FF,G,...,GC_avg_team_home_last_5,CA_avg_team_home_last_5,CV_avg_team_home_last_5,SG_avg_team_home_last_5,DD_avg_team_home_last_5,DP_avg_team_home_last_5,GS_avg_team_home_last_5,Pontos_avg_team_home_last_5,Preco_avg_team_home_last_5,game_points_avg_team_home_last_5
0,262.0,2014.0,1.0,0.923077,3.230769,0.000000,0.000000,0.307692,0.230769,0.000000,...,0.0,0.307692,0.000000,0.307692,0.076923,0.0,0.000000,1.992308,9.200000,1.000000
1,262.0,2014.0,2.0,1.153846,3.307692,0.000000,0.000000,0.307692,0.307692,0.000000,...,0.0,0.076923,0.076923,0.000000,0.076923,0.0,0.153846,1.346154,8.720769,0.000000
2,262.0,2014.0,3.0,1.384615,1.846154,0.230769,0.000000,0.230769,0.307692,0.307692,...,0.0,0.230769,0.000000,0.153846,0.038462,0.0,0.076923,3.446154,9.208462,2.000000
3,262.0,2014.0,4.0,1.214286,2.785714,0.000000,0.000000,0.142857,0.357143,0.000000,...,0.0,0.181319,0.038462,0.000000,0.074176,0.0,0.148352,0.737363,8.581099,0.000000
4,262.0,2014.0,5.0,0.785714,3.571429,0.000000,0.071429,0.357143,0.357143,0.000000,...,0.0,0.177656,0.000000,0.102564,0.097070,0.0,0.098901,2.971245,9.679212,1.333333


In [ ]:
df_player = pd.read_csv('../db/features_jogadores_final_calculadas.csv')

In [ ]:
df_player_team = df_player.merge(df_team[team_home_cols], how="left", on=["ClubeID", "Rodada", "ano"])

In [69]:
df_final_home = df_player_team_pos.loc[df_player_team_pos.ClubeID == df_player_team_pos.ClubeCasa]

In [70]:
df_final_away = df_player_team_pos.loc[df_player_team_pos.ClubeID == df_player_team_pos.ClubeVisitante]

In [72]:
df_final_home_enemy = df_final_home.merge(df_team[team_home_cols], how="left", 
                                          left_on=["ClubeVisitante", "Rodada", "ano"],
                                            right_on=["ClubeID","Rodada", "ano"])

In [73]:
df_final_away_enemy = df_final_away.merge(df_team[team_home_cols], how="left", 
                                          left_on=["ClubeCasa", "Rodada", "ano"],
                                            right_on=["ClubeID","Rodada", "ano"])

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'ScoutID', 'AtletaID',
       'Rodada', 'ClubeID_x', 'Participou', 'Posicao', 'Jogos',
       ...
       'GC_avg_team_home_last_5_y', 'CA_avg_team_home_last_5_y',
       'CV_avg_team_home_last_5_y', 'SG_avg_team_home_last_5_y',
       'DD_avg_team_home_last_5_y', 'DP_avg_team_home_last_5_y',
       'GS_avg_team_home_last_5_y', 'Pontos_avg_team_home_last_5_y',
       'Preco_avg_team_home_last_5_y', 'game_points_avg_team_home_last_5_y'],
      dtype='object', length=362)

In [76]:
df_final = pd.concat([df_final_home_enemy, df_final_away_enemy])

In [85]:
df_final = df_final.rename(columns=merge(map(lambda name: {name: name.strip('_y').replace('team', 'enemy')
                                       if name.endswith('_y') 
                                       else name.strip('_x')}, df_final.columns)))

In [87]:
df_final.shape

(37408, 362)

In [88]:
df_final.isna().sum().sort_values(ascending=False)

Apelido                             31116
Status                              31116
FD_avg_pos_home_last_5               8482
CA_avg_pos_home_last_10              8482
CV_avg_pos_home_last_10              8482
SG_avg_pos_home_last_10              8482
DD_avg_pos_home_last_10              8482
DP_avg_pos_home_last_10              8482
GS_avg_pos_home_last_10              8482
Pontos_avg_pos_home_last_10          8482
Preco_avg_pos_home_last_10           8482
game_points_avg_pos_home_last_10     8482
FS_avg_pos_home_last_5               8482
PE_avg_pos_home_last_5               8482
A_avg_pos_home_last_5                8482
FT_avg_pos_home_last_5               8482
FF_avg_pos_home_last_5               8482
FC_avg_pos_home_last_10              8482
G_avg_pos_home_last_5                8482
I_avg_pos_home_last_5                8482
PP_avg_pos_home_last_5               8482
RB_avg_pos_home_last_5               8482
FC_avg_pos_home_last_5               8482
GC_avg_pos_home_last_5            

In [89]:
df_final.to_csv('../db/final_features_calculated-2018-05-12.csv')

In [81]:
list(map(lambda name: name.strip('_y').replace('team', 'enemy') if name.endswith('_y') else name.strip('_x') ,df_final.columns))

['Unnamed: 0',
 'Unnamed: 0.1',
 'Unnamed: 0.1.1',
 'ScoutID',
 'AtletaID',
 'Rodada',
 'ClubeID',
 'Participou',
 'Posicao',
 'Jogos',
 'Pontos',
 'PontosMedia',
 'Preco',
 'PrecoVariacao',
 'FS',
 'PE',
 'A',
 'FT',
 'FD',
 'FF',
 'G',
 'I',
 'PP',
 'RB',
 'FC',
 'GC',
 'CA',
 'CV',
 'SG',
 'DD',
 'DP',
 'GS',
 'ano',
 'Apelido',
 'Status',
 'mes',
 'dia',
 'home_score',
 'away_score',
 'ClubeCasa',
 'ClubeVisitante',
 'Home',
 'result',
 'is_next_same',
 'target',
 'game_points',
 'FS_avg_last_20',
 'PE_avg_last_20',
 'A_avg_last_20',
 'FT_avg_last_20',
 'FD_avg_last_20',
 'FF_avg_last_20',
 'G_avg_last_20',
 'I_avg_last_20',
 'PP_avg_last_20',
 'RB_avg_last_20',
 'FC_avg_last_20',
 'GC_avg_last_20',
 'CA_avg_last_20',
 'CV_avg_last_20',
 'SG_avg_last_20',
 'DD_avg_last_20',
 'DP_avg_last_20',
 'GS_avg_last_20',
 'Pontos_avg_last_20',
 'Preco_avg_last_20',
 'game_points_avg_last_20',
 'FS_avg_last_10',
 'PE_avg_last_10',
 'A_avg_last_10',
 'FT_avg_last_10',
 'FD_avg_last_10',
 'FF_a

In [ ]:
rename(columns=merge(map(lambda name: {name: name.strip('_y').replace('team', 'enemy')
                                       if name.endswith('_y') 
                                       else name.strip('_x')}, aggCols))))

In [79]:
df_player_team_pos.shape

(39316, 298)

In [ ]:
df_player_team.shape

In [ ]:
df_player_team.head()

In [ ]:
df_player_team.isna().sum().sort_values()

In [ ]:
df_player_team.to_csv('../db/features_player_teams_final.csv')

In [6]:
df_player_team = pd.read_csv('../db/features_player_teams_final.csv')

/Users/henriquelopes/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (6,40,42) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
df_player_team.head()

,Unnamed: 0,Unnamed: 0.1,ScoutID,AtletaID,Rodada,ClubeID,Participou,Posicao,Jogos,Pontos,...,GC_avg_team_home_last_5,CA_avg_team_home_last_5,CV_avg_team_home_last_5,SG_avg_team_home_last_5,DD_avg_team_home_last_5,DP_avg_team_home_last_5,GS_avg_team_home_last_5,Pontos_avg_team_home_last_5,Preco_avg_team_home_last_5,game_points_avg_team_home_last_5
0,0,0,0.0,36443.0,1.0,285.0,True,gol,1.0,5.0,...,0.0,0.142857,0.0,0.357143,0.000000,0.0,0.000000,4.392857,11.063571,3.0
1,1,1,0.0,36443.0,2.0,285.0,True,gol,2.0,-3.0,...,0.0,0.285714,0.0,0.000000,0.000000,0.0,0.142857,2.892857,10.283571,1.0
2,2,2,15.0,36443.0,3.0,285.0,True,gol,3.0,-2.6,...,0.0,0.142857,0.0,0.178571,0.000000,0.0,0.035714,3.564286,10.961786,3.0
3,3,3,30.0,36443.0,4.0,285.0,True,gol,4.0,4.0,...,0.0,0.166667,0.0,0.119048,0.047619,0.0,0.047619,3.307143,11.015952,3.0
4,4,4,15.0,36443.0,5.0,285.0,True,gol,5.0,5.0,...,0.0,0.250000,0.0,0.178571,0.000000,0.0,0.071429,2.653571,10.208571,1.0


In [8]:
df_pos = df_player_team.groupby(['Posicao', 'ano', 'Rodada'], as_index=False)[columns_to_average].mean()

In [9]:
df_pos.head()

,Posicao,ano,Rodada,FS,PE,A,FT,FD,FF,G,...,GC,CA,CV,SG,DD,DP,GS,Pontos,Preco,game_points
0,ata,2014.0,1.0,1.687500,1.671875,0.078125,0.015625,0.562500,0.515625,0.125000,...,0.0,0.109375,0.000000,0.0,0.0,0.0,0.0,2.420313,9.750625,1.390625
1,ata,2014.0,2.0,1.430769,1.753846,0.138462,0.015385,0.415385,0.400000,0.169231,...,0.0,0.076923,0.000000,0.0,0.0,0.0,0.0,2.530769,8.520769,1.261538
2,ata,2014.0,3.0,1.583333,2.069444,0.069444,0.055556,0.500000,0.458333,0.194444,...,0.0,0.152778,0.000000,0.0,0.0,0.0,0.0,2.940278,8.924306,1.194444
3,ata,2014.0,4.0,1.446154,1.861538,0.076923,0.015385,0.323077,0.569231,0.215385,...,0.0,0.153846,0.046154,0.0,0.0,0.0,0.0,2.546154,8.584308,1.723077
4,ata,2014.0,5.0,1.528302,1.924528,0.075472,0.000000,0.358491,0.528302,0.075472,...,0.0,0.113208,0.018868,0.0,0.0,0.0,0.0,1.392453,8.874717,1.245283


In [10]:
df_pos.shape

(675, 24)

In [12]:
df_pos_avg = pd.DataFrame()
for num_rodadas in (5,10,20):
    df_pos_avg = pd.concat([getWindowOver(groupbyPos(df_player_team, columns_to_average),
                                          'Rodada', ['Posicao', 'ano'], columns_to_average,
                                         num_rodadas, 0, '_avg_pos'), df_pos_avg], axis=1)

In [13]:
df_pos_avg.head()

,FS_avg_pos_last_20,PE_avg_pos_last_20,A_avg_pos_last_20,FT_avg_pos_last_20,FD_avg_pos_last_20,FF_avg_pos_last_20,G_avg_pos_last_20,I_avg_pos_last_20,PP_avg_pos_last_20,RB_avg_pos_last_20,...,GC_avg_pos_last_5,CA_avg_pos_last_5,CV_avg_pos_last_5,SG_avg_pos_last_5,DD_avg_pos_last_5,DP_avg_pos_last_5,GS_avg_pos_last_5,Pontos_avg_pos_last_5,Preco_avg_pos_last_5,game_points_avg_pos_last_5
0,1.687500,1.671875,0.078125,0.015625,0.562500,0.515625,0.125000,0.515625,0.0,0.421875,...,0.0,0.109375,0.000000,0.0,0.0,0.0,0.0,2.420313,9.750625,1.390625
1,1.559135,1.712861,0.108293,0.015505,0.488942,0.457813,0.147115,0.503966,0.0,0.403245,...,0.0,0.093149,0.000000,0.0,0.0,0.0,0.0,2.475541,9.135697,1.326082
2,1.567201,1.831722,0.095344,0.028855,0.492628,0.457986,0.162892,0.484126,0.0,0.472534,...,0.0,0.113025,0.000000,0.0,0.0,0.0,0.0,2.630453,9.065233,1.282203
3,1.536939,1.839176,0.090739,0.025487,0.450240,0.485797,0.176015,0.474633,0.0,0.477477,...,0.0,0.123231,0.011538,0.0,0.0,0.0,0.0,2.609378,8.945002,1.392421
4,1.535212,1.856246,0.087685,0.020390,0.431890,0.494298,0.155906,0.485367,0.0,0.453680,...,0.0,0.121226,0.013004,0.0,0.0,0.0,0.0,2.365993,8.930945,1.362994


In [14]:
df_pos_avg.shape

(675, 63)

In [15]:
df_pos = pd.concat([df_pos, df_pos_avg], axis=1)

In [17]:
df_pos_home_avg = pd.DataFrame()
for num_rodadas in (5,10,20):
    df_pos_home_avg = pd.concat([getWindowOver(groupbyPos(filterHome(df_player_team, True), columns_to_average),
                                                'Rodada', ['Posicao', 'ano'], columns_to_average,
                                         num_rodadas, 0, '_avg_pos_home'), df_pos_home_avg], axis=1)

In [18]:
df_pos_home_avg.head()

,FS_avg_pos_home_last_20,PE_avg_pos_home_last_20,A_avg_pos_home_last_20,FT_avg_pos_home_last_20,FD_avg_pos_home_last_20,FF_avg_pos_home_last_20,G_avg_pos_home_last_20,I_avg_pos_home_last_20,PP_avg_pos_home_last_20,RB_avg_pos_home_last_20,...,GC_avg_pos_home_last_5,CA_avg_pos_home_last_5,CV_avg_pos_home_last_5,SG_avg_pos_home_last_5,DD_avg_pos_home_last_5,DP_avg_pos_home_last_5,GS_avg_pos_home_last_5,Pontos_avg_pos_home_last_5,Preco_avg_pos_home_last_5,game_points_avg_pos_home_last_5
0,1.764706,2.058824,0.117647,0.029412,0.647059,0.529412,0.117647,0.558824,0.0,0.441176,...,0.0,0.058824,0.0,0.0,0.0,0.0,0.0,2.767647,10.895294,1.588235
1,1.559772,1.771347,0.139469,0.030835,0.517078,0.425996,0.139469,0.489089,0.0,0.462524,...,0.0,0.061670,0.0,0.0,0.0,0.0,0.0,2.782211,8.820066,1.503795
2,1.539848,1.935284,0.119295,0.046872,0.511385,0.468208,0.171926,0.501498,0.0,0.527647,...,0.0,0.084973,0.0,0.0,0.0,0.0,0.0,3.038140,9.267939,1.327091
3,1.522074,1.998338,0.097284,0.042967,0.446039,0.468343,0.199257,0.508936,0.0,0.505111,...,0.0,0.079355,0.0,0.0,0.0,0.0,0.0,3.038762,9.178689,1.448444
4,1.548428,2.029440,0.085519,0.034373,0.433754,0.490059,0.167098,0.491764,0.0,0.457935,...,0.0,0.086561,0.0,0.0,0.0,0.0,0.0,2.616394,9.159874,1.297216


In [20]:
df_pos_home_avg.shape

(675, 63)

In [34]:
pos_home_cols = ['Posicao', 'Rodada', 'ano'] + list(df_pos_home_avg.columns)

In [28]:
df_pos_home = pd.concat([groupbyPos(filterHome(df_player_team, True), columns_to_average), df_pos_home_avg], axis=1)

In [24]:
df_pos_away_avg = pd.DataFrame()
for num_rodadas in (5,10,20):
    df_pos_away_avg = pd.concat([getWindowOver(groupbyPos(filterHome(df_player_team, False), columns_to_average),
                                                'Rodada', ['Posicao', 'ano'], columns_to_average,
                                         num_rodadas, 0, '_avg_pos_away'), df_pos_away_avg], axis=1)

In [25]:
df_pos_away_avg.head()

,FS_avg_pos_away_last_20,PE_avg_pos_away_last_20,A_avg_pos_away_last_20,FT_avg_pos_away_last_20,FD_avg_pos_away_last_20,FF_avg_pos_away_last_20,G_avg_pos_away_last_20,I_avg_pos_away_last_20,PP_avg_pos_away_last_20,RB_avg_pos_away_last_20,...,GC_avg_pos_away_last_5,CA_avg_pos_away_last_5,CV_avg_pos_away_last_5,SG_avg_pos_away_last_5,DD_avg_pos_away_last_5,DP_avg_pos_away_last_5,GS_avg_pos_away_last_5,Pontos_avg_pos_away_last_5,Preco_avg_pos_away_last_5,game_points_avg_pos_away_last_5
0,1.600000,1.233333,0.033333,0.000000,0.466667,0.500000,0.133333,0.466667,0.0,0.400000,...,0.0,0.166667,0.000000,0.0,0.0,0.0,0.0,2.026667,8.453333,1.166667
1,1.550000,1.616667,0.075490,0.000000,0.453922,0.485294,0.154902,0.512745,0.0,0.347059,...,0.0,0.127451,0.000000,0.0,0.0,0.0,0.0,2.157451,9.296667,1.142157
2,1.592157,1.695425,0.069935,0.009804,0.469281,0.441176,0.152288,0.459477,0.0,0.417647,...,0.0,0.143791,0.000000,0.0,0.0,0.0,0.0,2.191242,8.710817,1.241830
3,1.550178,1.657932,0.082754,0.007353,0.450446,0.497549,0.152094,0.435517,0.0,0.449599,...,0.0,0.168449,0.022727,0.0,0.0,0.0,0.0,2.160098,8.600007,1.340463
4,1.521624,1.667087,0.088425,0.005882,0.427023,0.494336,0.143898,0.474339,0.0,0.448568,...,0.0,0.156982,0.025589,0.0,0.0,0.0,0.0,2.096227,8.614524,1.427926


In [26]:
df_pos_away_avg.shape

(675, 63)

In [33]:
pos_away_cols = ['Posicao', 'Rodada', 'ano'] + list(df_pos_away_avg.columns)

In [30]:
df_pos_away = pd.concat([groupbyPos(filterHome(df_player_team, False), columns_to_average), df_pos_away_avg], axis=1)

In [31]:
df_pos_home.shape, df_pos_away.shape

((675, 87), (675, 87))

In [35]:
df_pos = df_pos.merge(df_pos_home[pos_home_cols], how="left", on=['Posicao', 'Rodada', 'ano'])

In [36]:
df_pos.shape

(675, 150)

In [37]:
df_pos = df_pos.merge(df_pos_away[pos_away_cols], how="left", on=['Posicao', 'Rodada', 'ano'])

In [38]:
for x,y in list(zip(pos_home_cols, pos_away_cols)):
    df_pos[x].fillna(df_pos[y], inplace=True)

In [39]:
df_pos.drop(list(set(pos_away_cols) - set(['Posicao', 'Rodada', 'ano'])),
        axis=1, inplace=True)

In [40]:
df_pos.head()

,Posicao,ano,Rodada,FS,PE,A,FT,FD,FF,G,...,GC_avg_pos_home_last_5,CA_avg_pos_home_last_5,CV_avg_pos_home_last_5,SG_avg_pos_home_last_5,DD_avg_pos_home_last_5,DP_avg_pos_home_last_5,GS_avg_pos_home_last_5,Pontos_avg_pos_home_last_5,Preco_avg_pos_home_last_5,game_points_avg_pos_home_last_5
0,ata,2014.0,1.0,1.687500,1.671875,0.078125,0.015625,0.562500,0.515625,0.125000,...,0.0,0.058824,0.0,0.0,0.0,0.0,0.0,2.767647,10.895294,1.588235
1,ata,2014.0,2.0,1.430769,1.753846,0.138462,0.015385,0.415385,0.400000,0.169231,...,0.0,0.061670,0.0,0.0,0.0,0.0,0.0,2.782211,8.820066,1.503795
2,ata,2014.0,3.0,1.583333,2.069444,0.069444,0.055556,0.500000,0.458333,0.194444,...,0.0,0.084973,0.0,0.0,0.0,0.0,0.0,3.038140,9.267939,1.327091
3,ata,2014.0,4.0,1.446154,1.861538,0.076923,0.015385,0.323077,0.569231,0.215385,...,0.0,0.079355,0.0,0.0,0.0,0.0,0.0,3.038762,9.178689,1.448444
4,ata,2014.0,5.0,1.528302,1.924528,0.075472,0.000000,0.358491,0.528302,0.075472,...,0.0,0.086561,0.0,0.0,0.0,0.0,0.0,2.616394,9.159874,1.297216


In [41]:
df_player_team_pos = df_player_team.merge(df_pos[pos_home_cols], how="left", on=["Posicao", "Rodada", "ano"])

In [42]:
df_player_team_pos.shape

(39316, 297)

In [43]:
df_player_team_pos.head()

,Unnamed: 0,Unnamed: 0.1,ScoutID,AtletaID,Rodada,ClubeID,Participou,Posicao,Jogos,Pontos,...,GC_avg_pos_home_last_5,CA_avg_pos_home_last_5,CV_avg_pos_home_last_5,SG_avg_pos_home_last_5,DD_avg_pos_home_last_5,DP_avg_pos_home_last_5,GS_avg_pos_home_last_5,Pontos_avg_pos_home_last_5,Preco_avg_pos_home_last_5,game_points_avg_pos_home_last_5
0,0,0,0.0,36443.0,1.0,285.0,True,gol,1.0,5.0,...,0.0,0.000,0.0,0.700000,0.600,0.0,0.500,4.150000,11.92800,1.600000
1,1,1,0.0,36443.0,2.0,285.0,True,gol,2.0,-3.0,...,0.0,0.000,0.0,0.500000,1.150,0.0,0.750,4.455000,14.25000,1.450000
2,2,2,15.0,36443.0,3.0,285.0,True,gol,3.0,-2.6,...,0.0,0.100,0.0,0.433333,1.000,0.0,0.900,3.153333,12.97300,1.333333
3,3,3,30.0,36443.0,4.0,285.0,True,gol,4.0,4.0,...,0.0,0.125,0.0,0.400000,1.075,0.0,0.900,3.135000,12.51175,1.425000
4,4,4,15.0,36443.0,5.0,285.0,True,gol,5.0,5.0,...,0.0,0.100,0.0,0.370000,1.160,0.0,0.945,3.195500,12.77615,1.315000


In [44]:
df_player_team_pos.isna().sum().sort_values()

Unnamed: 0                              0
Unnamed: 0.1                            0
FS_avg_home_last_5                   1908
PE_avg_home_last_5                   1908
A_avg_home_last_5                    1908
FT_avg_home_last_5                   1908
FD_avg_home_last_5                   1908
FF_avg_home_last_5                   1908
G_avg_home_last_5                    1908
I_avg_home_last_5                    1908
PP_avg_home_last_5                   1908
RB_avg_home_last_5                   1908
FC_avg_home_last_5                   1908
GC_avg_home_last_5                   1908
CA_avg_home_last_5                   1908
CV_avg_home_last_5                   1908
SG_avg_home_last_5                   1908
DD_avg_home_last_5                   1908
DP_avg_home_last_5                   1908
GS_avg_home_last_5                   1908
Pontos_avg_home_last_5               1908
Preco_avg_home_last_5                1908
game_points_avg_home_last_5          1908
FS_avg_team_home_last_20          

In [45]:
df_player_team_pos.to_csv('../db/features_player_teams_pos_final.csv')

In [48]:
df_player_team_pos = pd.read_csv('../db/features_player_teams_pos_final.csv')

/Users/henriquelopes/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (7,41,43) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [47]:
df_player_team_pos.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,ScoutID,AtletaID,Rodada,ClubeID,Participou,Posicao,Jogos,...,GC_avg_pos_home_last_5,CA_avg_pos_home_last_5,CV_avg_pos_home_last_5,SG_avg_pos_home_last_5,DD_avg_pos_home_last_5,DP_avg_pos_home_last_5,GS_avg_pos_home_last_5,Pontos_avg_pos_home_last_5,Preco_avg_pos_home_last_5,game_points_avg_pos_home_last_5
0,0,0,0,0.0,36443.0,1.0,285.0,True,gol,1.0,...,0.0,0.000,0.0,0.700000,0.600,0.0,0.500,4.150000,11.92800,1.600000
1,1,1,1,0.0,36443.0,2.0,285.0,True,gol,2.0,...,0.0,0.000,0.0,0.500000,1.150,0.0,0.750,4.455000,14.25000,1.450000
2,2,2,2,15.0,36443.0,3.0,285.0,True,gol,3.0,...,0.0,0.100,0.0,0.433333,1.000,0.0,0.900,3.153333,12.97300,1.333333
3,3,3,3,30.0,36443.0,4.0,285.0,True,gol,4.0,...,0.0,0.125,0.0,0.400000,1.075,0.0,0.900,3.135000,12.51175,1.425000
4,4,4,4,15.0,36443.0,5.0,285.0,True,gol,5.0,...,0.0,0.100,0.0,0.370000,1.160,0.0,0.945,3.195500,12.77615,1.315000
